# Dense dsm step by step from sensor images for Alice

This notebook correspond to the "DSM compute" part performed step by step.


### Imports

In [ ]:
# Notebook local imports

import os
import math
###
# Silent OTB info logs
os.environ['OTB_LOGGER_LEVEL']='WARNING'
import warnings
# Filter warnings
warnings.filterwarnings("ignore",category=UserWarning)
# import pretty print
import pprint 
pp = pprint.PrettyPrinter(indent=4)

# import external function for notebook
from notebook_helpers import get_full_data, show_data, save_data, get_dir_path, set_up_demo_inputs

In [ ]:
# Additional imports
from collections import OrderedDict
import numpy as np
import json
import matplotlib.pyplot as plt

In [ ]:
# CARS imports

# Applications
from cars.applications.application import Application
from cars.applications.grid_generation import grid_correction
from cars.applications.sparse_matching import sparse_matching_tools

# Pipelines
import cars.pipelines.sensor_to_dense_dsm.sensor_dense_dsm_constants as sens_cst
from cars.pipelines.sensor_to_dense_dsm import sensors_inputs
from cars.pipelines.sensor_to_dense_dsm import dsm_output

# Conf, core, orchestrator
from cars.conf import log_conf
from cars.core import inputs, preprocessing
from cars.core.utils import safe_makedirs
from cars.orchestrator import orchestrator
from cars.core.utils import make_relative_path_absolute

In [ ]:
# Show CARS version
from cars import __version__
print("CARS version used : {}".format(__version__))

---------

## Inputs/Outputs

### Define outputs

In [ ]:
# Modify with your own output path if needed
output_dir = os.path.join(get_dir_path(), "output_notebook")
print(output_dir)

## CARS Configuration 

More information for CARS configuration [here](https://cars.readthedocs.io/en/stable/user_guide/configuration.html)

### Define inputs

TODO : Ajouter les masques

In [ ]:
input_images_path = "/work/CAMPUS/users/sarrazine/outputs/bench_mns/data/refined"
img1_name = "IMG_P_001.tif"
mask1_name = "mask1.tif"
classif1_name = "classif1.tif"
img2_name = "IMG_P_002.tif"
mask2_name = "mask2.tif"
classif2_name = "classif2.tif"
roi_path = "/work/CAMPUS/users/sarrazine/outputs/bench_mns/out_refined/urban_area.shp"
roi_path="/work/CAMPUS/users/parasca/stage_alice/Montpellier/xt_ortho_IMG_001.tif"
prepare_path = "/work/CAMPUS/users/sarrazine/outputs/bench_mns/out_cars_0.5/prepare12/content.json"
srtm_dir = "/datalake/static_aux/MNT/SRTM_90m/"

In [ ]:
img1_path = os.path.join(input_images_path, img1_name)
img2_path = os.path.join(input_images_path, img2_name)

In [ ]:
xt_img1_path = os.path.join(output_dir, "xt_" + img1_name)
xt_img2_path = os.path.join(output_dir, "xt_" + img2_name)

ExtractROI

In [ ]:
import otbApplication as otb
def extract_roi(img_path, xt_img_path, roi_path, srtm_dir):
    app = otb.Registry.CreateApplication("ExtractROI")
    app.SetParameterString("in", img_path)
    app.SetParameterString("out", xt_img_path)
    app.SetParameterString("mode", "fit") 
    if os.path.splitext(roi_path)[1] == ".shp":
        app.SetParameterString("mode.fit.vect", roi_path)
    elif os.path.splitext(roi_path)[1] == ".tif":
        app.SetParameterString("mode.fit.im", roi_path)
    else:
        raise Exception("ROI extension not known")
    app.SetParameterString("elev.dem", srtm_dir)
    app.ExecuteAndWriteOutput()

In [ ]:
#extract_roi(img1_path, xt_img1_path, roi_path, srtm_dir)
#extract_roi(img2_path, xt_img2_path, roi_path, srtm_dir)

Get prepare information

In [ ]:
def get_prepare_infos(filepath, key):
    grid_correction = None
    disparity_range = None
    with open(prepare_path,'r') as f:
        prepare_conf = json.load(f)
        apriori_infos = prepare_conf["inputs"]["epipolar_a_priori"]
        if apriori_infos.get(key) is None :
            raise Exception("Pairing not available")
        grid_coeffecients = apriori_infos[key]["grid_correction"]
        disparity_range = apriori_infos[key]["disparity_range"]
    return grid_coeffecients, disparity_range

In [ ]:
grid_coefficients, disparity_range = get_prepare_infos(prepare_path, "img1_img2")

Create json configuration for CARS

In [ ]:
def configure_cars(img1,img2,srtm, roi, grid_coefs, disp_range):
    return {
                "sensors": {
                    "img1": {
                        "image": img1,
                        "geomodel": os.path.splitext(img1)[0]+".geom",
                        "color": img1, # On pourrait ajouter la couleur avec l'image P+XS
                        "no_data": 0,
                    },
                    "img2": {
                        "image": img2,
                        "geomodel": os.path.splitext(img2)[0]+".geom",
                        "no_data": 0,
                    },
                },
                "pairing": [["img1", "img2"]], # Mettre les combinaisons que l'on veut traiter
                "initial_elevation": srtm,
                "roi": roi, # Pour l'instant ça sert à rien
                "epipolar_a_priori": {
                  "img1_img2": {
                    "grid_correction": grid_coefs,
                    "disparity_range": disp_range,
                  }
                },
                "use_epipolar_a_priori": True,
            }


In [ ]:
inputs_conf = configure_cars(xt_img1_path,xt_img2_path,srtm_dir, roi_path, grid_coefficients, disparity_range)
pp.pprint(inputs_conf)

In [ ]:
updated_inputs_conf = sensors_inputs.sensors_check_inputs(inputs_conf)
pp.pprint(updated_inputs_conf)

---------

## Applications Init

#### GridGeneration

This application generates epipolar grids corresponding to sensor pair

In [ ]:
# TODO REMOVE

import importlib
import cars
importlib.reload(os)
importlib.reload(cars.applications.resampling.bicubic_resampling)
importlib.reload(cars.applications.dense_matching.census_mccnn_sgm)

In [ ]:
epipolar_grid_generation_application = Application("grid_generation")

#### Resampling

This application generates epipolar images from epipolar grids 

In [ ]:
resampling_application = Application("resampling")

#### DenseMatching

This application generates dense matches of stereo images pairs

In [ ]:
conf_pandora = OrderedDict([
    ('input',{'nodata_left': -9999, 'nodata_right': -9999}),
    ('pipeline',{
        'right_disp_map': {'method': 'accurate'},
        'matching_cost': {
            'matching_cost_method': 'census',
            'window_size': 5,
            'subpix': 1
        },
        'optimization': {
            'optimization_method': 'sgm',
            'overcounting': False,
            'penalty': {
                'P1': 8,
                'P2': 32,
                'p2_method': 'constant',
                'penalty_method': 'sgm_penalty'
            },
            'sgm_version': 'c++',
            'min_cost_paths': False,
            'use_confidence': False
        },
        'cost_volume_confidence': {
            'confidence_method': 'ambiguity',
            'eta_max': 0.7,
            'eta_step': 0.01,
           'indicator': ''
        },
        'disparity': {
            'disparity_method': 'wta',
            'invalid_disparity': np.nan
        },
        'refinement': {'refinement_method': 'vfit'},
        'filter': {'filter_method': 'median', 'filter_size': 3},
        'validation': {
            'validation_method': 'cross_checking',
            'cross_checking_threshold': 1.0
        }
    })
])

In [ ]:
conf_dense_matching = {'method': 'census_sgm', # A laisser on s'est fiche on surchage avec l'info au dessus, ce qui compte c'ets la conf de pandora
                       'loader': 'pandora',
                       'loader_conf': conf_pandora,
                      }

In [ ]:
dense_matching_application = Application("dense_matching",cfg=conf_dense_matching)

#### Show used application configuration

In [ ]:
# Example with dense matching application
dense_matching_application.print_config()

#### Triangulation

This application triangulates matches, in order to get each (X, Y, Z) point position

In [ ]:
triangulation_application = Application("triangulation")

#### PointCloudFusion

This application performs the fusion of epipolar points from pairs to a terrain point cloud

In [ ]:
pc_fusion_application = Application("point_cloud_fusion")

#### PointCloudOutliersRemoving : small components

This application removes outliers points. The method used is the "small components removing"

In [ ]:
conf_outlier_removing_small_components = {"method": "small_components", "activated": True}
pc_outlier_removing_small_comp_application = Application("point_cloud_outliers_removing", cfg=conf_outlier_removing_small_components)

#### PointCloudOutliersRemoving : statistical

This application removes outliers points. The method used is the "statistical removing"

In [ ]:
conf_outlier_removing_small_statistical = {"method": "statistical", "activated": True}
pc_outlier_removing_stats_application = Application("point_cloud_outliers_removing", cfg=conf_outlier_removing_small_statistical)


#### PointCloudRasterization

This application performs the rasterization of a terrain point cloint.

In [ ]:
conf_rasterization = { 
    "method": "simple_gaussian",
    "dsm_radius": 3,
    "sigma": 0.3
}
rasterization_application = Application("point_cloud_rasterization", cfg=conf_rasterization)

### Create orchestrator


In [ ]:
# Use sequential mode in notebook
orchestrator_conf = {"mode": "mp", "nb_workers": 10} # On reste en séquentiel pour l'instant
cars_orchestrator = orchestrator.Orchestrator(orchestrator_conf=orchestrator_conf, out_dir=output_dir)

In [ ]:

def compute_cell(orchestrator, list_cars_ds):
    
    # add cars datasets to save lists
    for cars_ds in list_cars_ds:
        orchestrator.add_to_replace_lists(cars_ds)
        
    # trigger computation and replacement
    orchestrator.breakpoint()

---------

## Run pipeline step by step from sensors to DSM

### Sensors images generation

From input configuration "inputs" seen before

In [ ]:
_, sensor_image_left, sensor_image_right = sensors_inputs.generate_inputs(updated_inputs_conf)[0]

### Grid Generation : epipolar grid generation

In [ ]:


grid_left, grid_right = epipolar_grid_generation_application.run(
    sensor_image_left,
    sensor_image_right,
    orchestrator=cars_orchestrator,
    srtm_dir=updated_inputs_conf[sens_cst.INITIAL_ELEVATION],
    default_alt=updated_inputs_conf[sens_cst.DEFAULT_ALT],
    geoid_path=updated_inputs_conf[sens_cst.GEOID],
)

### Correct right grid

Correction coefficients for the right grid (computed during the prepare step)

In [ ]:

grid_coefficients

Generate corrected right grid

In [ ]:
def apply_grid_correction(grid, conf): 
    coefficients = conf["epipolar_a_priori"]["img1_img2"]["grid_correction"]
    coefs_x = coefficients[:3]
    coefs_x.append(0.0)
    coefs_y = coefficients[3:6]
    coefs_y.append(0.0)
    grid_coeffs = (
    np.array(coefs_x).reshape((2, 2)),
    np.array(coefs_y).reshape((2, 2)),
              )
    # Correct grid right with provided epipolar a priori
    return grid_correction.correct_grid(grid, grid_coeffs)

In [ ]:
corrected_grid_right = apply_grid_correction(grid_right, updated_inputs_conf)

### Define disparity interval

In [ ]:
dmin, dmax = disparity_range
print(f"disp min = {dmin:.2f}")
print(f"disp max = {dmax:.2f}")

### Compute margins used in dense matching, with corresponding disparity min and max

In [ ]:
dense_matching_margins, disp_min, disp_max = dense_matching_application.get_margins(
    grid_left, disp_min=dmin, disp_max=dmax)

### Resampling : epipolar images generation


In [ ]:
epipolar_image_left, epipolar_image_right = resampling_application.run(
    sensor_image_left,
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    orchestrator=cars_orchestrator,
    margins=dense_matching_margins,
    optimum_tile_size=(
        dense_matching_application.get_optimal_tile_size(
            disp_min, 
            disp_max,
            cars_orchestrator.cluster.checked_conf_cluster[
                "max_ram_per_worker"
            ],
        )
    ),
    add_color=True,
)
# Compute cell              
compute_cell(cars_orchestrator, [epipolar_image_left, epipolar_image_right])


### Show epipolar image

In [ ]:
data_image_left = get_full_data(epipolar_image_left, "im")
data_mask_left = get_full_data(epipolar_image_left, "msk")
data_image_right = get_full_data(epipolar_image_right, "im")
data_mask_right = get_full_data(epipolar_image_right, "msk")

In [ ]:
data_mask_left[np.where(data_mask_left !=0)]

In [ ]:
def show_epilolar_images(img_left, mask_left, img_right, mask_right, fig_size=8):
    clip_percent = 5
    vmin_left = np.percentile(img_left,clip_percent)
    vmax_left = np.percentile(img_left,100-clip_percent)
    vmin_right = np.percentile(img_right,clip_percent)
    vmax_right = np.percentile(img_right,100-clip_percent)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(fig_size, 1.05 * fig_size / 2), subplot_kw={'aspect': 1})
    axes[0].set_title("Left image")
    axes[0].imshow(img_left, cmap="gray", interpolation='spline36', vmin=vmin_left, vmax=vmax_left)
    axes[0].imshow(np.ma.masked_where(mask_left == 0, mask_left), cmap='tab10', alpha=0.5)
    axes[0].axhline(len(img_left)/2., color='red')
    axes[1].set_title("Right image")
    axes[1].imshow(img_right, cmap="gray", interpolation='spline36', vmin=vmin_right, vmax=vmax_right)
    axes[1].imshow(np.ma.masked_where(mask_right == 0, mask_right), cmap='tab10', alpha=0.5)
    axes[1].axhline(len(img_right)/2., color='red')
    fig.tight_layout()

In [ ]:
show_epilolar_images(data_image_left, data_mask_left, data_image_right, data_mask_right)

### Dense Matching: compute disparities with pandora

In [ ]:
epipolar_disparity_map_left, epipolar_disparity_map_right = dense_matching_application.run(
    epipolar_image_left,
    epipolar_image_right,
    orchestrator=cars_orchestrator,
    disp_min=disp_min,
    disp_max=disp_max,
)
               
# Compute cell              
compute_cell(cars_orchestrator, [epipolar_disparity_map_left, epipolar_disparity_map_right])



#### Show full disparity map

In [ ]:
data_disparity = get_full_data(epipolar_disparity_map_left, "disp")
show_data(data_disparity, figsize=(4, 4))

 Compute epsg

In [ ]:
epsg = preprocessing.compute_epsg(
    sensor_image_left, 
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    triangulation_application.get_geometry_loader(),
    orchestrator=cars_orchestrator,
    srtm_dir=updated_inputs_conf[sens_cst.INITIAL_ELEVATION],
    default_alt=updated_inputs_conf[sens_cst.DEFAULT_ALT],
    disp_min=disp_min,
    disp_max=disp_max
)

### Triangulation : triangulate matches

In [ ]:
epipolar_points_cloud_left, epipolar_points_cloud_right = triangulation_application.run(
    sensor_image_left,
    sensor_image_right,
    epipolar_image_left,
    epipolar_image_right,
    grid_left,
    corrected_grid_right,
    epipolar_disparity_map_left,
    epipolar_disparity_map_right,
    epsg,
    orchestrator=cars_orchestrator,
    uncorrected_grid_right=grid_right,
    geoid_path=updated_inputs_conf[sens_cst.GEOID],
    disp_min=disp_min,
    disp_max=disp_max,
)
# Compute cell              
compute_cell(cars_orchestrator, [epipolar_points_cloud_left, epipolar_points_cloud_right])

 #### Compute terrain bounding box

In [ ]:
current_terrain_roi_bbox = preprocessing.compute_terrain_bbox(
    updated_inputs_conf[sens_cst.INITIAL_ELEVATION],
    updated_inputs_conf[sens_cst.DEFAULT_ALT],
    updated_inputs_conf[sens_cst.GEOID],
    sensor_image_left,
    sensor_image_right,
    epipolar_image_left,
    grid_left,
    corrected_grid_right,
    epsg,
    triangulation_application.get_geometry_loader(),
    resolution=rasterization_application.get_resolution(),
    disp_min=disp_min,
    disp_max=disp_max,
    orchestrator=cars_orchestrator
)
terrain_bounds, optimal_terrain_tile_width = preprocessing.compute_terrain_bounds(
    [current_terrain_roi_bbox],
    resolution=rasterization_application.get_resolution()
)

#### Transform point cloud to terrain point cloud

In [ ]:
merged_points_clouds = pc_fusion_application.run(
    [epipolar_points_cloud_left],
    [epipolar_points_cloud_right],
    terrain_bounds,
    epsg,
    orchestrator=cars_orchestrator,
    margins=rasterization_application.get_margins(),
    optimal_terrain_tile_width=optimal_terrain_tile_width
)
# Compute cell              
compute_cell(cars_orchestrator, [merged_points_clouds])

### Point Cloud Outlier Removing : remove points with small components removing method

In [ ]:
filtered_sc_merged_points_clouds = pc_outlier_removing_small_comp_application.run(
    merged_points_clouds,
    orchestrator=cars_orchestrator,
)    
# Compute cell              
compute_cell(cars_orchestrator, [filtered_sc_merged_points_clouds])

### Point Cloud Outlier Removing: remove points with statistical removing method

In [ ]:
filtered_stats_merged_points_clouds = pc_outlier_removing_stats_application.run(
    filtered_sc_merged_points_clouds,
    orchestrator=cars_orchestrator,
)
# Compute cell              
compute_cell(cars_orchestrator, [filtered_stats_merged_points_clouds])

### Rasterization : rasterize point cloud

In [ ]:
dsm = rasterization_application.run(
    filtered_stats_merged_points_clouds,
    epsg,
    orchestrator=cars_orchestrator
)
# Compute cell              
compute_cell(cars_orchestrator, [dsm])

### Show DSM


In [ ]:
data_dsm = get_full_data(dsm, "hgt")
show_data(data_dsm, figsize=(4, 4), mode="dsm")

### Show ortho image

In [ ]:
data_ortho = get_full_data(dsm, "img")
show_data(data_ortho, figsize=(4, 4), mode='image')


### Save DSM

In [ ]:
save_data(dsm, os.path.join(output_dir, "dsm.tif"), "hgt")